In [1]:
# Dependencies
import pandas as pd
import math
import folium
import re
from geopy.geocoders import ArcGIS
from geopy.exc import GeocoderTimedOut
from geopy.point import Point

Load the data

In [2]:
df = pd.read_excel('Sample Dataset.xlsx', header=0)

Distance column
New coordinates
New Address
Use cases

In [3]:
# Haversine formula
def haversine(coord1, coord2):
    R = 6371.0  # radius of Earth in kilometers
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

In [4]:
geolocator = ArcGIS(user_agent="myGeocoder")

def geoencode_arcgis(details):
    locations = []
    for detail in details:
        full_address = ', '.join(detail)
        try:
            location = geolocator.geocode(full_address)
            if location is not None:
                locations.append((location.latitude, location.longitude))
        except GeocoderTimedOut:
            print("GeocoderTimedOut: geocode failed on input %s with message %s"%(full_address, "GeocoderTimedOut"))
    return locations

def reverse_geocode(location):
    point = Point(location[0], location[1])
    try:
        address = geolocator.reverse(point)
        if address is not None:
            # Use regex to extract the pincode from the address string
            pincode = re.findall(r'\d{6}', address.address)[0]
            return address.address, pincode
    except GeocoderTimedOut:
        print("GeocoderTimedOut: reverse geocode failed on input %s with message %s"%(location, "GeocoderTimedOut"))
    return None, None

def update_coordinates(row):
    name = str(row['NAME'])
    address_base = str(row['ADDRESS'])
    area = str(row['AREA'])
    city = str(row['CITY'])
    pincode = str(row['pin_code'])
    original_coordinates = (row['lattitude'], row['longitude'])

    # List of different levels of address details
    address_details = [
        [name, pincode],
        [name, address_base, pincode],
        [name, area, pincode],
        [name, address_base, area, city, pincode]
    ]
    
    locations = geoencode_arcgis(address_details)

    if locations:
        # Calculate the distances for each geoencoded location and find the minimum
        distances = [haversine(original_coordinates, loc) for loc in locations]
        min_distance_index = distances.index(min(distances))
        best_location = locations[min_distance_index]
        best_distance = distances[min_distance_index]
        
        # Reverse geocoding on the best location
        best_reverse_address, new_pincode = reverse_geocode(best_location)
        
        return best_location[0], best_location[1], best_distance, best_reverse_address, new_pincode

    return original_coordinates[0], original_coordinates[1], None, None, None


Driver Code

In [5]:
# Apply the function to update coordinates to the dataframe
df['New LATTITUDE'], df['New LONGITUDE'], df['Distance (km)'], df['New Address'], df['New Pincode'] = zip(*df.apply(update_coordinates, axis=1))
df['New LATTITUDE'], df['New LONGITUDE'] = df['New LONGITUDE'], df['New LATTITUDE']
df

,NAME,ADDRESS,AREA,CITY,pin_code,longitude,lattitude,New LATTITUDE,New LONGITUDE,Distance (km),New Address,New Pincode
0,Cyber Hub,"21, DLF Tower 10th Rd, DLF Cyber City, DLF Pha...",Haryana,Gurgaon,122022.0,77.0891,28.4950,77.08988,28.49725,0.261543,"Raghvendra Marg, Gurugram, Haryana 122002, IND",122002
1,Ambience Mall,"Ground floor, Ambience Mall",Haryana,Gurgaon,NaN,77.0960,28.5055,77.09573,28.50582,0.044296,"Ambience Mall Road, Gurugram, Haryana 122010, IND",122010
2,The Grand Venice,"Plot No SH3, Site IV, near Pari Chowk",Uttar Pradesh,Greater Noida,201308.0,77.5263,28.4525,77.52623,28.45264,0.017005,"Mastiii Zone, Surajpur Site 4, Greater Noida, ...",201308
3,Kingdom of Dreams,Great Indian Nautanki Pvt. Ltd. Auditorium Com...,Haryana,Gurgaon,122001.0,77.0689,28.4679,77.06818,28.46846,0.093972,"Leisure Valley Park Road, Gurugram, Haryana 12...",122007
4,The Oberoi,"443, Shankar Chowk Rd, opposite Metro Station,...",Haryana,Gurugram,122016.0,77.0882,28.5022,77.08879,28.50255,0.069560,"Amaranta, Udyog Vihar, Gurugram, Haryana, 1220...",122016
5,The Leela Palace,"National Highway 8, Ambience Island, DLF Phase...",Haryana,Gurugram,122002.0,77.0965,28.5054,77.09676,28.50561,0.034507,"Ambience Mall Road, Gurugram, Haryana 122010, IND",122010


Final Output

In [6]:
# Convert the dataframe to a XLXS file
df.to_excel('Sample dataset Updated.xlsx', index=False)

Cross Verification

In [7]:
# Plotting function
def plot_map(row, threshold):
    # Base map
    m = folium.Map(location=(row['lattitude'], row['longitude']), zoom_start=15)

    # Original coordinates
    original_coordinates = (row['lattitude'], row['longitude'])

    # Add a marker for the original coordinates
    folium.Marker(
        location=original_coordinates,
        popup='Original',
        icon=folium.Icon(color='green', icon='info-sign')
    ).add_to(m)

    # Get the updated coordinates from the dataframe
    updated_coordinates = (row['New LONGITUDE'], row['New LATTITUDE'])
    
    # Add a marker for the updated coordinates
    folium.Marker(
        location=updated_coordinates,
        popup=f"Updated: {updated_coordinates}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

    # Plot a threshold radius circle using folium.Circle for updated coordinates
    folium.Circle(
        location=updated_coordinates,
        radius=threshold*1000,  # radius in meters
        color='red',
        fill=True,
        fill_color='red'
    ).add_to(m)

    return m

In [8]:
# Plot the map for a sample row
threshold = 1 
plot_map(df.iloc[0], threshold)

In [9]:
# Load the updated dataset
df_insights = pd.read_excel('Sample Dataset Updated.xlsx', header=0)

# Number of dealers with distance greater than threshold
df_insights[df_insights['Distance (km)'] > threshold].shape[0]

# Print the dealers with distance greater than threshold
df_insights[df_insights['Distance (km)'] > threshold]

,NAME,ADDRESS,AREA,CITY,pin_code,longitude,lattitude,New LATTITUDE,New LONGITUDE,Distance (km),New Address,New Pincode
